# Actividad 2
### Librerías

In [ ]:
%matplotlib inline 

import matplotlib.pyplot as plt
import cv2

In [ ]:
imagen = cv2.imread('pictures/book1.bmp', cv2.IMREAD_GRAYSCALE)
plt.imshow(imagen, cmap='gray')